In [ ]:
# Trabalho MO833

In [ ]:
## Imports

In [35]:
import sys
sys.path.append('../..')
import os
import yaml
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from app.cli.modules.cluster import get_cluster_config_db, get_cluster_manager
from clap.utils import float_time_to_string, path_extend
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor

In [ ]:
## Setup Managers

In [2]:
configuration_db = get_config_db()
cluster_config_db = get_cluster_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
private_path = node_manager.private_path
cluster_manager = get_cluster_manager()

In [ ]:
## Check instances types

In [ ]:
!cat ~/.clap/configs/instances.yaml

In [ ]:
## Setar o objeto das configurations das instancias

In [3]:
type_c4x_instance_info = configuration_db.instance_descriptors['type-c4x']
type_c5x_instance_info = configuration_db.instance_descriptors['type-c5x']

In [ ]:
## Iniciando nós

In [4]:
started_node_ids_c4 = node_manager.start_node(type_c4x_instance_info, count=1)

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Starting 1 type-c4x instances (timeout 600 seconds)] *********************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Tagging instances] *******************************************************
changed: [localhost] => (item={'id': 'i-099eeb8da47067677', 'name': 'AudraHartley-a77a77bc'})

PLAY RECAP *********************************************************************
localhost                 

In [5]:
started_node_ids_c5x = node_manager.start_node(type_c5x_instance_info, count=1)

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Starting 1 type-c5x instances (timeout 600 seconds)] *********************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Tagging instances] *******************************************************
changed: [localhost] => (item={'id': 'i-03dbc747f2d217893', 'name': 'JoeKeenan-e580702e'})

PLAY RECAP *********************************************************************
localhost                  : 

In [ ]:
## Verify instances details and generate ID list (You need to wait until the instance be reachable)

In [17]:
alive_nodes_c4 = node_manager.is_alive(started_node_ids_c4)
alive_nodes_c5x = node_manager.is_alive(started_node_ids_c5x)
for node_id, alive_flag in alive_nodes_c4.items():
    alive_str = 'alive' if alive_flag == True else 'not alive'
    print(f"{node_id} --> {alive_str}.")
for node_id, alive_flag in alive_nodes_c5x.items():
    alive_str = 'alive' if alive_flag == True else 'not alive'
    print(f"{node_id} --> {alive_str}.")

a77a77bcb55a42009e1864652193bac6 --> alive.
e580702ef32049298d5f0a3356dce8f5 --> alive.


In [18]:
node_id_list = []
for node in node_manager.get_all_nodes():
    # Can be accessed with '.' operator
    node_id = node.node_id
    node_id_list.append(node_id)
    print('---------')
    print(f"Node Id: {node_id}, created at {float_time_to_string(node.creation_time)}; Status: {node.status};  IP: {node.ip}")
    print('---------')
node_id_list

---------
Node Id: a77a77bcb55a42009e1864652193bac6, created at 30-06-21 15:59:14; Status: reachable;  IP: 35.171.164.95
---------
---------
Node Id: e580702ef32049298d5f0a3356dce8f5, created at 30-06-21 15:59:55; Status: reachable;  IP: 52.90.139.235
---------


['a77a77bcb55a42009e1864652193bac6', 'e580702ef32049298d5f0a3356dce8f5']

In [ ]:
## Tag nodes

In [12]:
tag_c4 = {
    'node-type': 'c4'
}
tag_c5x = {
    'node-type': 'c5x'
}
node_ids_with_tags_c4 = node_manager.add_tags([started_node_ids_c4[0]], tag_c4)
node_ids_with_tags_c5x = node_manager.add_tags([started_node_ids_c5x[0]], tag_c5x)

In [ ]:
## Setup de commands to use with clap API
Aqui nós podemos configurar qualquer parameto da aplicação, a seguir as linhas de comando que foram executadas nos nós

In [13]:
command_to_execute_c4 = """ 
cd high-res-distributed &&
sudo docker run --rm --network=host -v=$(pwd):/root super-res:latest python -m torch.distributed.launch --nproc_per_node=2 --nnodes=1 --node_rank=0 --master_addr="172.17.0.1" --master_port=1234 esrgan.py --dataset_name="img_align_celeba_test" --n_cpu=2 --n_epochs=1
"""
command_to_execute_c5x = """
cd high-res-distributed &&
sudo docker run --rm --network=host -v=$(pwd):/root super-res:latest python -m torch.distributed.launch --nproc_per_node=4 --nnodes=1 --node_rank=0 --master_addr="172.17.0.1" --master_port=1234 esrgan.py --dataset_name="img_align_celeba_test" --n_cpu=4 --n_epochs=1
"""

In [ ]:
## Get config objects

In [14]:
nodes_config_c4 = node_manager.get_nodes_by_id(started_node_ids_c4)
nodes_config_c5x = node_manager.get_nodes_by_id(started_node_ids_c5x)

In [ ]:
## Run the command in each instance

In [15]:
executor_c4 = SSHCommandExecutor(command_to_execute_c4, nodes_config_c4, private_path)
result_c4 = executor_c4.run()

In [16]:
#Results
for node_id, res in result_c4.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    # resut is a dataclass, we can convert to a dictionary
    res_dict_c4 = asdict(res)
    print('-----')
    # Dump dictionary in YAML format
    print(yaml.dump(res_dict_c4, indent=4, sort_keys=True))

Node id a77a77bcb55a42009e1864652193bac6, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'The module torch.distributed.launch is deprecated and going to be removed in future.Migrate
    to torch.distributed.run

    '
- 'WARNING:torch.distributed.run:--use_env is deprecated and will be removed in future
    releases.

    '
- ' Please read local_rank from `os.environ(''LOCAL_RANK'')` instead.

    '
- 'INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:

    '
- '  entrypoint       : esrgan.py

    '
- '  min_nodes        : 1

    '
- '  max_nodes        : 1

    '
- '  nproc_per_node   : 2

    '
- '  run_id           : none

    '
- '  rdzv_backend     : static

    '
- '  rdzv_endpoint    : 172.17.0.1:1234

    '
- '  rdzv_configs     : {''rank'': 0, ''timeout'': 900}

    '
- '  max_restarts     : 3

    '
- '  monitor_interval : 5

    '
- '  log_dir          : None

    '
- '  metrics_cfg      : {}

  

In [19]:
executor_c5x = SSHCommandExecutor(command_to_execute_c5x, nodes_config_c5x, private_path)
result_c5x = executor_c5x.run()

In [20]:
#Results
for node_id, res in result_c5x.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    # resut is a dataclass, we can convert to a dictionary
    res_dict_c5x = asdict(res)
    print('-----')
    # Dump dictionary in YAML format
    print(yaml.dump(res_dict_c5x, indent=4, sort_keys=True))

Node id e580702ef32049298d5f0a3356dce8f5, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'The module torch.distributed.launch is deprecated and going to be removed in future.Migrate
    to torch.distributed.run

    '
- 'WARNING:torch.distributed.run:--use_env is deprecated and will be removed in future
    releases.

    '
- ' Please read local_rank from `os.environ(''LOCAL_RANK'')` instead.

    '
- 'INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:

    '
- '  entrypoint       : esrgan.py

    '
- '  min_nodes        : 1

    '
- '  max_nodes        : 1

    '
- '  nproc_per_node   : 4

    '
- '  run_id           : none

    '
- '  rdzv_backend     : static

    '
- '  rdzv_endpoint    : 172.17.0.1:1234

    '
- '  rdzv_configs     : {''rank'': 0, ''timeout'': 900}

    '
- '  max_restarts     : 3

    '
- '  monitor_interval : 5

    '
- '  log_dir          : None

    '
- '  metrics_cfg      : {}

  

In [ ]:
## Fetch results usando Clapp

In [21]:
role_manager = get_role_manager()

In [ ]:
for role_name, role_info in role_manager.roles.items():
    # Convert role_info to a dict
    role_dict = asdict(role_info)
    print('------')
    print(f"Role: {role_name}")
    # Print role
    print(f"{yaml.dump(role_dict, indent=4)}")

In [ ]:
## Adicionando as roles aos nós

In [22]:
added_nodes = role_manager.add_role('experiment_result', node_id_list)
print(f"Role experiment result was added to {len(added_nodes)}")

Role experiment result was added to 2


In [ ]:
## Executando o playbook nas instancias

In [25]:
#Os repositorios devem existir previamente
copy_vars_c4 = {
    'output': path_extend('~/Documents/results/c4x/'),
}
copy_vars_c5 = {
    'output': path_extend('~/Documents/results/c5x/'),
}

In [26]:
playbook_result = role_manager.perform_action('experiment_result', 'result', started_node_ids_c4, extra_args=copy_vars_c4)


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [a77a77bcb55a42009e1864652193bac6]

TASK [Find files to copy] ******************************************************
ok: [a77a77bcb55a42009e1864652193bac6]

TASK [Copy output files to /home/gabrielribeiro/Documents/results/c4/] *********
changed: [a77a77bcb55a42009e1864652193bac6] => (item={'path': '/home/ubuntu/high-res-distributed/mo833-trabalho/experiments/TEST_EXPERIMENT_ID/1625079915.4443498/PIs_logs/TEST_NODE-TEST_EXPERIMENT_ID-1625079915.4443638.out', 'mode': '0644', 'isdir': False, 'ischr': False, 'isblk': False, 'isreg': True, 'isfifo': False, 'islnk': False, 'issock': False, 'uid': 0, 'gid': 0, 'size': 467, 'inode': 256023, 'dev': 51713, 'nlink': 1, 'atime': 1625080380.6255286, 'mtime': 1625079974.0861416, 'ctime': 1625079974.0861416, 'gr_name': 'root', 'pw_name': 'root', 'wusr': True, 'rusr': True, 'xusr': Fal

In [27]:
playbook_result = role_manager.perform_action('experiment_result', 'result', started_node_ids_c5x, extra_args=copy_vars_c5)


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [e580702ef32049298d5f0a3356dce8f5]

TASK [Find files to copy] ******************************************************
ok: [e580702ef32049298d5f0a3356dce8f5]

TASK [Copy output files to /home/gabrielribeiro/Documents/results/c5/] *********
changed: [e580702ef32049298d5f0a3356dce8f5] => (item={'path': '/home/ubuntu/high-res-distributed/mo833-trabalho/experiments/TEST_EXPERIMENT_ID/1625080072.5985966/PIs_logs/TEST_NODE-TEST_EXPERIMENT_ID-1625080072.5986068.out', 'mode': '0644', 'isdir': False, 'ischr': False, 'isblk': False, 'isreg': True, 'isfifo': False, 'islnk': False, 'issock': False, 'uid': 0, 'gid': 0, 'size': 323, 'inode': 257141, 'dev': 66305, 'nlink': 1, 'atime': 1625080115.1333375, 'mtime': 1625080132.2779272, 'ctime': 1625080132.2779272, 'gr_name': 'root', 'pw_name': 'root', 'wusr': True, 'rusr': True, 'xusr': Fal

In [ ]:
#Nessa parte é necessario configurar os Paths.

In [29]:
instance_costs = {
  "c4x": 0.199,
  "c5x": 0.170
}

node_info_c4 = {
    "node_id": f"{started_node_ids_c4[0]}",
    "node_type": "c4x"
}

node_info_c5 = {
    "node_id": f"{started_node_ids_c5x[0]}",
    "node_type": "c5x"
}

#Adjust this to your location
pi_logs_dir = '/home/gabrielribeiro/Documents/results/'
node_list = [node_info_c4, node_info_c5]
node_list

[{'node_id': 'a77a77bcb55a42009e1864652193bac6', 'node_type': 'c4x'},
 {'node_id': 'e580702ef32049298d5f0a3356dce8f5', 'node_type': 'c5x'}]

In [ ]:
#Implement Reporter

In [30]:
class Reporter:
    def get_metrics(self, node_list_info, experiment_id, pi_logs_dir, instance_costs):

        dict_metrics = {} 
        node_count = 0
        iteration_count = 0
        
        for node in node_list:
            file_data = []
            file_dir = f'{pi_logs_dir}{node["node_type"]}/'
            file_list = os.listdir(file_dir)

            #Read file
            for file in file_list:
                if os.path.isfile(os.path.join(file_dir, file)):
                    with open(os.path.join(file_dir, file),'r') as f:
                        for line in f:
                            values_line = line.split(",")
                            file_data.append(values_line)

            #Extract Data
            for line in file_data:
                if "[MO833] Rank" in line:
                    node_count_file = int(line[line.index("[MO833] Rank") + 1]) + 1
                    if(node_count_file > node_count):
                        node_count = node_count_file
                if file_data.index(line) == 0:
                    value = line[2].split(":")
                    value_filtered = value[1].replace("\n", "")
                    value_filtered = value_filtered.strip()
                    initialization_time = float(value_filtered)

                if "Epoch time" in line:
                    epoch_time = float(line[line.index("Epoch time") + 1])

                if "Iteration" in line:
                    iteration_count_file = float(line[line.index("Iteration") + 1]) + 1
                    if(iteration_count_file > iteration_count):
                        iteration_count = iteration_count_file

            avarage_iteration_duration_second = (epoch_time) / (iteration_count * node_count * 1.0)
            #use node (CPU) count
            seconds_with_one_dolar = ((1.0 / instance_costs[node["node_type"]]) * 3600) * node_count 
            avarage_iterations_per_dolar =  (seconds_with_one_dolar - initialization_time) / (avarage_iteration_duration_second)
            dict_metrics[node["node_type"]] = avarage_iterations_per_dolar

        return dict_metrics
            
    def terminated(self, cluster_id: str, experiment_id: str) -> bool:
        #Esse método não foi implementado porque utilizei experimentos bem pequenos
        #Fiz isso para garantir ter recursos suficientes na AWS
        pass
    def fetch_results(self, node_list: [], experiment_id: str, output_dir: str):
        #Esse método ja foi executado no notebook, então seria só portar o código para ca
        #Como as informações aqui ja estão salvas não é necessário utilizar esse método
        pass

In [38]:
#Implement Optimizer
class Optimizer:
    def run(self, cluster_id: str, experiment_id: str, metrics):
        #Como toda a execução do experimento ja foi realizada essa classe no fim somente retorna a máquina mais eficiente para o treinamento
        best_value = 0
        for instance_type, instance_value in metrics.items():
            if instance_value > best_value:
                best_instance = instance_type
                best_value = instance_value
        print(f"A melhor instancia para esse problema é {best_instance} comm uma média de {best_value} iterations por dolar")

In [40]:
reporter = Reporter()
optimizer = Optimizer()

In [36]:
#-> Antes de Rodar o reporter é necessário apagar manualmente o arquivo abaixo do path onde foram carregados os resultados.
#Nome do arquivo: TEST_NODE-TEST_EXPERIMENT_ID-1624306733.317315.out
#Razão: a imagem utilizada nas máquinas contem um lixo de execução de algum teste que realizei
result_dict = reporter.get_metrics(node_list, 123, pi_logs_dir, instance_costs)
result_dict

{'c4x': 12162.851743451454, 'c5x': 55507.49240632235}

In [41]:
optimizer.run("sample-cluster", "test-id", result_dict)

A melhor instancia para esse problema é c5x comm uma média de 55507.49240632235 iterations por dolar
